# IBM Applied Data Science Capstone Project

## Location identification for New Mall in Bengaluru, India.

### Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#Analysis)
* [Conclusion](#conclusion)

### Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a **Shopping Mall**. Specifically, this report will be targeted to stakeholders interested in opening an **Shopping Mall** in **Bengaluru**, India.

Since there are lots of malls in Bengaluru we will try to detect **locations that are not already crowded with malls**. We would  prefer locations **as close to city center as possible**.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

### Data <a name="data"></a>

In order to work on our porblem, we will need the following data:
* Names of neighborhoods in Bengaluru.
* The geographical co-ordinates of each neighborhood.
* All shopping malls in Bengaluru

In order to acquire these data, we have used the following sources:
* Foursquare API
* Google Maps Reverse Geocoding

## Methodology <a name="methodology"></a>

In this project we will direct our efforts in identifying areas with less Shopping Mall density. We will focus only in areas within the city only and will not consider any neighboring sub-urban areas. We start but scrapping the list of all neighborhoods in Bengaluru and performing the necessary data preprocessing. Then we use a combination of Geocoder library and Foursquare API to acquire geographical coordinates and venues in the neighborhoods. Lastly, we use K-means algorithm from Scikit-Learn's cluster library to perform cluster analysis.

#### Importing Necessary Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
#Acquiring neighborhoods list and making a dataframe
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore'
wiki_table = pd.read_html(url)

The above operations returns a list of tables. The list 'wiki_table' has a lenght of 12 as shown below. Now, we will assign it to different dataframes and then merge it into one dataframe.

In [3]:
print(len(wiki_table))

12


In [4]:
#Keeping on the necessary tables
wiki_table = pd.concat(wiki_table)


In [5]:
l = pd.Series([i for i in range(79)])
wiki_table.set_index(l)

,0,1,2,Image,Name,Places adjacent to List of neighbourhoods in Bangalore,Places adjacent to List of neighbourhoods in Bangalore.1,Summary,Unnamed: 2
0,NaN,NaN,NaN,NaN,Cantonment area,NaN,NaN,The Cantonment area in Bangalore was used as a...,NaN
1,NaN,NaN,NaN,NaN,Domlur,NaN,NaN,"Formerly part of the Cantonment area, Domlur h...",NaN
2,NaN,NaN,NaN,NaN,Indiranagar,NaN,NaN,Indiranagar is a sought-after residential and ...,NaN
3,NaN,NaN,NaN,NaN,Jeevanbheemanagar,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,Malleswaram,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,Pete area,NaN,NaN,Established by Kempe Gowda I at the time of cr...,NaN
6,NaN,NaN,NaN,NaN,Sadashivanagar,NaN,NaN,Sadashivanagar is an upscale neighbourhood in ...,NaN
7,NaN,NaN,NaN,NaN,Seshadripuram,NaN,NaN,Seshadripuram was established in 1892 to reduc...,NaN
8,NaN,NaN,NaN,NaN,Shivajinagar,NaN,NaN,Shivajinagar is one of the older areas of the ...,NaN
9,NaN,NaN,NaN,NaN,Ulsoor,NaN,NaN,Ulsoor (or Halasuru) is one of the oldest area...,NaN


In [6]:
#Keeping on the names column
final_df = pd.DataFrame(wiki_table['Name'])
final_df.dropna(inplace=True)
final_df = final_df.set_index(l[0:65])

### Getting the coordinates of each neighborhood

In [7]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bengaluru, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [8]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in final_df["Name"].tolist() ]

In [9]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# merge the coordinates into the original dataframe
final_df['Latitude'] = df_coords['Latitude']
final_df['Longitude'] = df_coords['Longitude']

In [10]:
final_df.head()

,Name,Latitude,Longitude
0,Cantonment area,25.48268,81.861570
1,Domlur,12.94329,77.656020
2,Indiranagar,13.03006,77.495260
3,Jeevanbheemanagar,12.96601,77.657670
4,Malleswaram,13.00632,77.568405


### Creating a map of Bengaluru with along with the Neighborhoods

In [11]:
# get the coordinates of Bengaluru
address = 'Bengaluru, India'

geolocator = Nominatim(user_agent="BangaloreMall")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bengaluru, India: {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bengaluru, India: 12.9791198, 77.5912997.


In [12]:
# create map of Toronto using latitude and longitude values
map_bg = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Name']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bg)  
    
map_bg

### Using Foursquare API to explore the neighborhoods

In [13]:
# define Foursquare Credentials and Version
CLIENT_ID = 'TYI0NAJJUGPMUO21AU5S4YYDTJXUJ3OQLITE2NMEVKV1IZTQ' # Removed for confidentiality
CLIENT_SECRET = 'C2QOXGVSQFVT4ME4QMFUGPO1LKBKIEYESSOT5DEFWPLHUWBS' # Removed for confidentiality
VERSION = '20180605' # Foursquare API version

### Exploring the top 100 venues that are within a radius of 2000 meters.

In [19]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Name']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
   
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3401, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Cantonment area,25.48268,81.86157,Axis Bank ATM,25.475950,81.854752,ATM
1,Cantonment area,25.48268,81.86157,Chechi Canteen,25.494819,81.867502,Fast Food Restaurant
2,Cantonment area,25.48268,81.86157,Axis Bank ATM,25.470130,81.867350,ATM
3,Cantonment area,25.48268,81.86157,Prayag Station,25.468158,81.867459,Train Station
4,Cantonment area,25.48268,81.86157,Jagram,25.469582,81.851829,Dessert Shop


In [21]:
#Checking how many venues were returned for each neighorhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Anjanapura,5,5,5,5,5,5
Arekere,76,76,76,76,76,76
BTM Layout,88,88,88,88,88,88
Banashankari,100,100,100,100,100,100
Banaswadi,51,51,51,51,51,51
Basavanagudi,100,100,100,100,100,100
Basaveshwaranagar,35,35,35,35,35,35
Begur,20,20,20,20,20,20
Bellandur,87,87,87,87,87,87


### Finding out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 214 uniques categories.


In [23]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### Analysis <a name= "Analysis"></a>

In [24]:
# one hot encoding
bg_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bg_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bg_onehot.columns[-1]] + list(bg_onehot.columns[:-1])
bg_onehot = bg_onehot[fixed_columns]

print(bg_onehot.shape)
bg_onehot.head()

(3401, 215)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Cantonment area,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Cantonment area,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Cantonment area,1,0,0,0,0,0,0,

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
bg_grouped = bg_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(bg_grouped.shape)
bg_grouped

(65, 215)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Anjanapura,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.200000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.2,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.200000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000

In [26]:
len(bg_grouped[bg_grouped["Shopping Mall"] > 0])

29

#### Create a new DataFrame for Shopping Mall data only

In [27]:
bg_mall = bg_grouped[["Neighborhoods","Shopping Mall"]]
bg_mall.head()

,Neighborhoods,Shopping Mall
0,Anjanapura,0.000000
1,Arekere,0.026316
2,BTM Layout,0.011364
3,Banashankari,0.010000
4,Banaswadi,0.000000


### Clustering Neighborhoods

Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [28]:
# set number of clusters
kc = 3

bg_clustering = bg_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kc, random_state=0).fit(bg_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0])

In [29]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
bg_merged = bg_mall.copy()

# add clustering labels
bg_merged["Cluster Labels"] = kmeans.labels_
bg_merged.head()

,Neighborhoods,Shopping Mall,Cluster Labels
0,Anjanapura,0.000000,0
1,Arekere,0.026316,2
2,BTM Layout,0.011364,0
3,Banashankari,0.010000,0
4,Banaswadi,0.000000,0


In [30]:
# merge bg_grouped with bg_data to add latitude/longitude for each neighborhood
bg_merged = bg_merged.join(final_df.set_index('Name'), on='Neighborhoods')

print(bg_merged.shape)
bg_merged.head() # check the last columns!

(65, 5)


,Neighborhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Anjanapura,0.000000,0,12.858110,77.559090
1,Arekere,0.026316,2,12.885680,77.596680
2,BTM Layout,0.011364,0,12.914880,77.610040
3,Banashankari,0.010000,0,12.922310,77.569880
4,Banaswadi,0.000000,0,13.019647,77.654694


In [31]:
# sort the results by Cluster Labels
print(bg_merged.shape)
bg_merged.sort_values(["Cluster Labels"], inplace=True)
bg_merged

(65, 5)


,Neighborhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Anjanapura,0.000000,0,12.858110,77.559090
28,Kalyan Nagar,0.000000,0,12.968020,77.521140
29,Kamakshipalya,0.000000,0,12.986990,77.524840
31,Kengeri,0.000000,0,12.908700,77.487140
63,Yelahanka,0.000000,0,13.099310,77.592590
33,Kothnur,0.000000,0,13.064340,77.648550
35,Kumaraswamy Layout,0.000000,0,12.898190,77.559270
37,Madiwala,0.010000,0,12.920520,77.620900
42,Mathikere,0.000000,0,13.032350,77.558660
43,Nagarbhavi,0.000000,0,12.956240,77.509360


#### Finally, let's visualize the resulting clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kc)
ys = [i+x+(i*x)**2 for i in range(kc)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bg_merged['Latitude'], bg_merged['Longitude'], bg_merged['Neighborhoods'], bg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 0

In [33]:
bg_merged.loc[bg_merged['Cluster Labels'] == 0]

,Neighborhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Anjanapura,0.000000,0,12.858110,77.559090
28,Kalyan Nagar,0.000000,0,12.968020,77.521140
29,Kamakshipalya,0.000000,0,12.986990,77.524840
31,Kengeri,0.000000,0,12.908700,77.487140
63,Yelahanka,0.000000,0,13.099310,77.592590
33,Kothnur,0.000000,0,13.064340,77.648550
35,Kumaraswamy Layout,0.000000,0,12.898190,77.559270
37,Madiwala,0.010000,0,12.920520,77.620900
42,Mathikere,0.000000,0,13.032350,77.558660
43,Nagarbhavi,0.000000,0,12.956240,77.509360


#### Cluster 1

In [34]:
bg_merged.loc[bg_merged['Cluster Labels'] == 1]

,Neighborhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
25,Jalahalli,0.166667,1,13.0545,77.52658


#### Cluster 2

In [35]:
bg_merged.loc[bg_merged['Cluster Labels'] == 2]

,Neighborhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
62,Whitefield,0.023256,2,12.975230,77.752380
1,Arekere,0.026316,2,12.885680,77.596680
22,Hulimavu,0.017544,2,12.880640,77.601470
58,Varthur,0.035714,2,12.943480,77.747030
56,Ulsoor,0.020000,2,12.989080,77.627950
55,Shivajinagar,0.020000,2,12.987200,77.604010
30,Kammanahalli,0.015625,2,13.014300,77.636850
40,Malleswaram,0.020000,2,13.006320,77.568405
39,Mahalakshmi Layout,0.016667,2,13.016350,77.544810
34,Krishnarajapuram,0.030303,2,13.000390,77.683680


### Conclusion <a name = "conclusion"></a>

Most of the shopping malls are concentrated in the central area of Bengaluru city, with the highest number in cluster 0 and cluster 2. On the other hand, cluster 1 has just one shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 0 and 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Lastly, property developers are advised to avoid neighborhoods in cluster 0 and 2 which already have high concentration of shopping malls and suffering from intense competition.